In [1]:
# Install the Generative AI library
#!pip install -q google-generativeai
#! pip uninstall -y google-generativeai
! pip install -q google-genai

import io, wave, google.genai as genai
from google.genai import types
from google.colab import userdata
from IPython.display import Audio, display, Markdown

! pip install ipywebrtc
from ipywebrtc import AudioRecorder, CameraStream
from google.colab import output

!pip install pydub
from pydub import AudioSegment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 6.6 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [2]:
API_KEY=userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=API_KEY)

model_name = 'gemini-2.5-flash'
chat = client.chats.create(model=model_name)

txt = input("Enter your message: ")
response = chat.send_message(txt)
display(Markdown(response.text))

Enter your message: hi


Hi there! How can I help you today?

In [ ]:
# Enable custom widget manager
output.enable_custom_widget_manager()

# Create a stream and a recorder (this creates the visible widget)
camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)
display(recorder)

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [ ]:
recorded_audio_data = recorder.audio.value
if recorded_audio_data: print(f"Successfully accessed recorded audio data of size: {len(recorded_audio_data)} bytes")
else: print("No audio data recorded yet. Please record some audio using the widget above.")


Successfully accessed recorded audio data of size: 52460 bytes


In [ ]:
# Convert the webm audio data to mp3
audio_segment = AudioSegment.from_file(io.BytesIO(recorded_audio_data), format="webm")
mp3_audio_data = audio_segment.export(format="mp3").read()

In [ ]:
Ttxt = "Transcribe the spoken words in this audio file verbatim."
Tresponse = client.models.generate_content(model=model_name, contents=[Ttxt, types.Part.from_bytes(data=mp3_audio_data, mime_type='audio/mp3')])
print("--- Transcription Result ---")
display(Markdown(Tresponse.text))

--- Transcription Result ---


Hello, how are you?

In [ ]:
A_prompt = ""
A_response = client.models.generate_content(model=model_name, contents=[A_prompt,  Tresponse.text ] )
print("\n--- output ---")
display(Markdown(A_response.text))


--- output ---


Hello! I'm doing well, thank you for asking.

How are you today?

**Everything now**

In [13]:
output.enable_custom_widget_manager()
camera = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=camera)
display(recorder)

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [14]:
recorded_audio_data = recorder.audio.value
audio_segment = AudioSegment.from_file(io.BytesIO(recorded_audio_data), format="webm")
mp3_audio_data = audio_segment.export(format="mp3").read()

Ttxt = "Transcribe the spoken words in this audio file verbatim."
T_response = client.models.generate_content(model=model_name, contents=[Ttxt, types.Part.from_bytes(data=mp3_audio_data, mime_type='audio/mp3')])
print("--- Transcription Result ---")
display(Markdown(T_response.text))

A_prompt = ""
A_response = client.models.generate_content(model=model_name, contents=[A_prompt,  T_response.text ] )
print("\n--- output ---")
display(Markdown(A_response.text))

--- Transcription Result ---


Get news from CNN.


--- output ---


To get the most up-to-the-minute news directly from CNN, I recommend:

1.  **Visiting their official website:** [CNN.com](https://www.cnn.com/)
2.  **Downloading the CNN app:** Available for iOS and Android devices.
3.  **Watching CNN on TV:** Check your local cable or satellite provider.

These are the best ways to get real-time updates, breaking news, and in-depth coverage directly from the source.

As an AI, I don't have a live, constantly refreshing feed of breaking news. However, I can help you in a couple of ways:

*   I can search for **recent top headlines or stories from CNN** if you'd like me to look them up.
*   If you have a **specific topic you're interested in** (e.g., U.S. politics, world news, business), I can try to find relevant CNN articles or summaries.

Just let me know what you're looking for!

In [15]:
# Text To Speech

def play_audio_from_response(response):
    """Writes the raw audio data from a Gemini response to a WAV file and plays it in Colab."""
    try:
        # 1. Extract the raw audio blob from the response
        blob = response.candidates[0].content.parts[0].inline_data
        audio_data = blob.data

        # 2. Define the output file name
        fname = 'generated_audio.wav'

        # 3. Write the raw PCM audio data to a properly formatted WAV file
        with wave.open(fname, "wb") as wf:
            wf.setnchannels(1)           # Mono channel
            wf.setsampwidth(2)           # 16-bit sample width (PCM)
            wf.setframerate(24000)       # 24kHz sampling rate
            wf.writeframes(audio_data)

        # 4. Display the audio player in Colab
        display(Audio(fname, autoplay=True))

    except Exception as e:
        print(f"Error processing audio response: {e}")
        print("Ensure you are using a '-tts' model and have API access for audio generation.")

TTS_MODEL = "gemini-2.5-flash-preview-tts"
txt2S = "Say cheerfully: Have a wonderful day! I hope you build something amazing with the Gemini API."
txt2S = A_response.text

S_response = client.models.generate_content(model=TTS_MODEL, contents=txt2S, config=types.GenerateContentConfig(response_modalities=["AUDIO"] ))

print("Generating and playing audio...")
play_audio_from_response(S_response)

Output hidden; open in https://colab.research.google.com to view.